# Combination of Anglescan Analysis
Taken from combining together the analysis routines worked out in result_concatenation_4

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import collections
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.magnum.utils as mgut
import flopter.core.fitters as fts
import flopter.core.constants as c

In [2]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_datasets = '/home/jleland/data/externy/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'phobos_test'
# path_to_analysed_datasets = 'test'
# path_to_analysed_datasets = 'analysed_3_downsampled'
# path_to_analysed_datasets = 'analysed_4'
path_to_analysed_datasets = 'analysed_4_downsampled'
# path_to_analysed_datasets = 'analysed_5_downsampled'
os.chdir(path_to_datasets)

In [3]:
magnum_probes = lp.MagnumProbes()
print(magnum_probes.probe_position.items())

dict_items([('l', 6), ('s', -4), ('b', -14), ('r', -24)])


## Load adc file metadata

In [4]:
os.chdir('/home/jleland/data/external/magnum/')
# os.chdir('/home/jleland/data/externy/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
# print(meta_data_ds)

---
## Whole Analysis Function


In [5]:
def run_whole_anglescan_analysis(indices, probe_settings, global_fit_settings):
    angle_scan_ds = mgut.get_dataset_from_indices(indices, anglescan_fl=True, average_direction_fl=False,
                                                  path_to_analysed_datasets=path_to_analysed_datasets)
    fit_df = mgut.fit_multi_magnum_ds(angle_scan_ds, probes_settings=probe_settings, threshold=None, 
                                      plot_fl=False, **global_fit_settings)
    combined_ds = mgut.combine_fit_ds(fit_df, angle_scan_ds)
    return combined_ds

---
### Indices for different Anglescans

In [6]:
# Hydrogen Shots
# super_title = 'H shot @ 0.8T'
indices_08T_H = [41,42,43,44,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63]
ps_08T_H = {
    ('S', 'L'): {'mode': 3, 'trimming_vals': (0.16, 0.16, 0.02), 'sat_region': -25},
    ('B', 'R'): {'mode': 3, 'trimming_vals': (0.16, 0.16, 0.02), 'sat_region': -25},
}
ps_08T_H_alternate = {
    ('S', 'L'): {'mode': 3, 'trimming_vals': (0.16, 0.16, 0.02), 'sat_region': -25, 'fitter':fts.SimpleIVFitter()},
}

# super_title = 'H shot @ 1.2T'
indices_12T_H = [320,321,322,323,324,325,327,328,329,330,331,336,337,338,339,340,341,342,343,344,345,346]
ps_12T_H = {
    ('S', 'L'): {'mode': 3, 'trimming_vals': (0.2, 0.0, 0.02), 'sat_region': -50},
    ('B'): {'mode': 3, 'trimming_vals': (0.2, 0.1, 0.02), 'sat_region': -50},
    ('R'): {'mode': 3, 'trimming_vals': (0.2, 0.3, 0.02), 'sat_region': -50},
}

# super_title = 'H shot @ 1.5T'
indices_15T_H = [490,491,492,493,494,495,496,497,498,499,500,503,504,505,506,507,509,510,511,512,513,514]
ps_15T_H = {
    ('S', 'L', 'B'): {'mode': 0, 'trimming_vals': (0.3, 0.0, 0.02), 'sat_region': -50},
    ('R'): {'mode': 3, 'trimming_vals': (0.3, 0.2, 0.02), 'sat_region': -50},
}

# Helium Shots
# super_title = 'He shot @ 0.8T'
indices_08T_He = [357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386]
ps_08T_He = {
    ('S'): {'mode': 0, 'trimming_vals': (0.3, 0.0, 0.02), 'sat_region': -60},
    ('B'): {'mode': 0, 'trimming_vals': (0.3, 0.3, 0.02), 'sat_region': -60},
    ('R'): {'mode': 3, 'trimming_vals': (0.3, 0.15, 0.02), 'sat_region': -60},
}

# super_title = 'He shot @ 1.2T'
indices_12T_He = [245,246,247,248,249,250,251,252,254,255,256,261,262,263,264,265,266,267,268,269,270,271]
ps_12T_He = {
    ('S', 'L'): {'mode': 0, 'trimming_vals': (0.16, 0.0, 0.02), 'sat_region': -75},
    ('B'): {'mode': 2, 'sat_region': -65, 'minimise_temp_fl':False}   
}

# Deuterium Shot
# super_title = 'D shot @ 0.8T'
indices_08T_D = [462,463,464,465,466,467,468,469,470,471,472,475,476,477,478,479,480,481,482,483,484,485]
ps_08T_D = {
    ('S', 'L'): {'sat_region': -65, 'minimise_temp_fl':False, 'temp_from_phi_fl': False}, 
    ('B'): {'mode': 0, 'trimming_vals': (0.1, 0.1, 0.02), 'sat_region': -30}
}


In [9]:
plasma_settings = [
    (0.8, 'H'),
#     (1.2, 'H'),
#     (1.5, 'H'),
#     (0.8, 'He'),
#     (1.2, 'He'),
]

fit_inputs = {
    (0.8, 'H'): (indices_08T_H, ps_08T_H, {}),
#     (1.2, 'H'): (indices_12T_H, ps_12T_H, {}),
#     (1.5, 'H'): (indices_15T_H, ps_15T_H, {}),
#     (0.8, 'He'): (indices_08T_He, ps_08T_He, {'mass': 2}),
#     (1.2, 'He'): (indices_12T_He, ps_12T_He, {'mass': 2}),
}

---
## Aggregation!
The below cell runs all analyses and compiles the outputted 'ds_combined's into a list


In [10]:
fit_datasets = {}
for plasma_settings, fit_settings in fit_inputs.items():
    indices, probe_settings, other_settings = fit_settings
    fit_dataset = run_whole_anglescan_analysis(indices, probe_settings, other_settings)
    fit_datasets[plasma_settings] = fit_dataset

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/projects/flopter/flopter/core/lputils.py:202: RuntimeWarning: divide by zero encountered in double_scalars
  a = ((c_1 + (c_2 / np.tan(alpha))) / np.sqrt(np.sin(alpha))) * (lambda_D / (L + g))
/home/jleland/coding/projects/flopter/flopter/core/lputils.py:202: RuntimeWarning: divide by zero encountered in double_scalars
  a = ((c_1 + (c_2 / np.tan(alpha))) / np.sqrt(np.sin(alpha))) * (lambda_D / (L + g))
/home/jleland/coding/projects/flopter/flopter/core/lputils.py:202: RuntimeWarning: divide by zero encountered in double_scalars
  a = ((c_1 + (c_2 / np.tan(alpha))) / np.sqrt(np.sin(alpha))) * (lambda_D / (L + g))
/home/jleland/coding/projects/flopter/flopter/core/lputils.py:202: RuntimeWarning: divide by zero encountered in double_scalars
  a = ((c_1 + (c_2 / np.tan(alpha))) / np.sqrt(n

RuntimeError occured in stage 2. 
Original error: Optimal parameters not found: The maximum number of function evaluations is exceeded.


In [11]:
fit_datasets[(0.8, 'H')].sel(probe='S').shot_number

<xarray.DataArray 'shot_number' (tilt: 11)>
array([42. , 43. , 44. , 45. , 46. , 47. , 48.5, 49.5, 50.5, 51.5, 52.5])
Coordinates:
    probe    <U1 'S'
  * tilt     (tilt) float64 -0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0

In [12]:
def interpolate_ts_position(dataset, uncertainty=1.5, offset=0):
    magnum_probes = lp.MagnumProbes()
    
    ts_temp = dataset.ts_temperature.mean(['probe', 'tilt'])
    ts_d_temp = dataset.ts_temperature.std(['probe', 'tilt'])
    ts_dens = dataset.ts_density.mean(['probe', 'tilt'])
    ts_d_dens = dataset.ts_density.std(['probe', 'tilt'])

    ts_probe_temps = {}
    ts_probe_denss = {}
    for probe, probe_pos in magnum_probes.probe_position.items():
        probe_pos += offset
        probe_pos_limits = [probe_pos, probe_pos - uncertainty, probe_pos + uncertainty]
        ts_probe_temps[probe.upper()] = ts_temp.interp(ts_radial_pos=probe_pos_limits).values
        ts_probe_denss[probe.upper()] = ts_dens.interp(ts_radial_pos=probe_pos_limits).values
        
    return ts_probe_temps, ts_probe_denss


## Plotting!

In [13]:
# The default colours from pyplot
# tableau_palette = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple',]
# markers = ['.', '+', 'x', '^', '*', 'o']
probe_colours = {
    'L': '#4477AA',
    'S': '#EE6677',
    'B': '#228833',
    'R': '#CCBB44'
}
probe_markers = {
    'L': 's',
    'S': '.',
    'B': '^',
    'R': 'p',
}
tableau_palette = ['tab:blue', 'tab:green', 'tab:red', 'black', 'tab:purple', 'tab:grey', 'tab:orange']
cb_palette = ['#4477AA', '#66CCEE', '#228833', '#CCBB44', '#EE6677', '#AA3377', '#BBBBBB']
markers = ['.', '+', 'x', '^', '*', 'o', 's']
global_options = {'fontsize':12, }

probes = ['L', 'S', 'B', 'R']
# probe_colours = {probes[i]: tableau_palette[i] for i in range(len(probes))}


In [14]:
import itertools

def plot_probe_anglescans(probe, fit_datasets, parameters=('temp', 'dens')):
    fig, ax = plt.subplots(2, constrained_layout=True)
    labels = []
    handles = []

    for i, (plasma, fit_ds) in enumerate(fit_datasets.items()):
        probe_ds = fit_ds.sel(probe=probe)

        ts_probe_temps, ts_probe_denss = interpolate_ts_position(fit_ds)

        label = f'{plasma[0]} - {plasma[1]}'
        ts_label = f'TS at {label}'

        handle = ax[0].errorbar('tilt', 'temp', yerr='d_temp', data=probe_ds, fmt=markers[i], label=label, 
                                color=tableau_palette[i])
        ax[1].errorbar('tilt', 'dens', yerr='d_dens', fmt=markers[i], data=probe_ds, color=tableau_palette[i])


        # Plot interpolated TS value over the top
        ts_handle = ax[0].axhline(y=ts_probe_temps[probe][0], color=tableau_palette[i], linewidth=1, 
                                        linestyle='dashed', label=ts_label)
        ax[1].axhline(y=ts_probe_denss[probe][0], color=tableau_palette[i], linewidth=1, 
                      linestyle='dashed')

        handles.append(handle)
        labels.append(label)
        labels.append(ts_label)
        handles.append(ts_handle)    


    ax[0].set_ylabel('T / eV')
    ax[0].set_yscale('log')
    # ax[0].set_ylim(0, 15)

    ax[1].set_yscale('log')
    ax[1].set_ylabel(r'$n_e$ / m$^{-3}$')
    ax[1].set_xlabel(r'probe tilt / $^{\circ}$')


    fig.legend(handles, labels, loc='centre left', bbox_to_anchor=(0.8,0.5))
    fig.set_constrained_layout_pads(hspace=0.0, wspace=3.0)
    fig.suptitle(f'Anglescan Fits for All {probe} Probe Shots')

In [15]:
import importlib
importlib.reload(lp)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

In [16]:
def plot_all_tiltscan_fitparams(plasma_settings, ax=None, phi_temp_fl=False, probes=('S', 'L', 'B'), 
                            fontsize=18):
    if ax is None:
        fig, ax = plt.subplots(2, 3, sharex=True, constrained_layout=True, figsize=[15,10])
    else:
        fig = ax[0,0].figure
        
    ds = fit_datasets[plasma_settings]
    
    if phi_temp_fl:
        ds['d_temp_phi'] = np.abs(ds['temp_phi'] - ds['temp_fit_phi'])

    
    labels = []
    handles = []

    for i, probe in enumerate(probes):
        probe_label = probe[0]

        ds_probe = ds.sel(probe=probe)

        ts_probe_temps, ts_probe_denss = interpolate_ts_position(ds)
        probe_dims = magnum_probes[probe_label]

        dummy_tilt = np.linspace(0.00001, 11.0, 1100)
    #     dummy_tilt = np.linspace(89.99, 78.99, 1101)
        calced_a = lp.calc_sheath_expansion_coeff(
            ts_probe_temps[probe_label][0],
            ts_probe_denss[probe_label][0],
            probe_dims.get_2d_probe_length(),
            probe_dims.g, 
            np.radians(dummy_tilt),
            c_1=0.9,
            c_2=0.6,
        )
        calced_new_a = lp.calc_new_sheath_expansion_param(
            ts_probe_temps[probe_label][0],
            ts_probe_denss[probe_label][0],
            probe_dims.get_2d_probe_length(),
            probe_dims.g, 
            np.radians(dummy_tilt),
            probe_dims.d_perp,
            0.0 if not probe_dims.is_angled() else probe_dims.theta_p, 
            c_1=0.9,
            c_2=0.6,
        )

        handle = ax[0][0].errorbar('tilt', 'temp', yerr='d_temp', data=ds_probe, label=probe, fmt='.', 
                                   color=probe_colours[probe])
        labels.append(probe)
        handles.append(handle)
        if phi_temp_fl:
            label = r'$T_e$ from $\phi$ on {}'.format(probe)
            handle = ax[0][0].errorbar('tilt', 'temp_phi', yerr='d_temp_phi', data=ds_probe, mfc='none', 
                                       label=label, linestyle='none', marker='^', color=probe_colours[probe])
            labels.append(label)
            handles.append(handle)

        ax[0][1].errorbar('tilt', 'v_f', yerr='d_v_f', data=ds_probe, fmt='.', color=probe_colours[probe])
        ax[0][2].plot('tilt', 'reduced_chi2', '.', data=ds_probe, color=probe_colours[probe])



        ax[1][0].errorbar('tilt', 'dens', yerr='d_dens', data=ds_probe, fmt='.', color=probe_colours[probe])
        ax[1][1].errorbar('tilt', 'isat', yerr='d_isat', data=ds_probe, fmt='.', color=probe_colours[probe])
        ax[1][2].errorbar('tilt', 'a', yerr='d_a', data=ds_probe, fmt='.', color=probe_colours[probe])
        calced_a_label = r'$a_{{{probe}, calc}}$'.format(probe=probe)
        handle = ax[1][2].errorbar(dummy_tilt, calced_a, label=calced_a_label, fmt='-', color=probe_colours[probe], 
                                   linewidth=0.8, alpha=0.6)
        ax[1][2].errorbar(dummy_tilt, calced_new_a, label=calced_a_label, fmt=':', color=probe_colours[probe], 
                          linewidth=0.8, alpha=0.6)
        labels.append(calced_a_label)
        handles.append(handle)

        # Plot the ts values at the probe positions using interpolaton
        handle = ax[0][0].axhline(y=ts_probe_temps[probe_label][0], color=probe_colours[probe], linewidth=1, 
                                  linestyle='dashed', label=f'TS at {probe}')
        ax[1][0].axhline(y=ts_probe_denss[probe_label][0], color=probe_colours[probe], linewidth=1, 
                         linestyle='dashed')

        labels.append(f'TS at {probe}')
        handles.append(handle)    


    ax[0,0].set_ylabel('T / eV', fontsize=fontsize)
    ax[0,1].set_ylabel(r'$V_f$ (V)', fontsize=fontsize)
    ax[0,2].set_ylabel(r'$\chi^2_{\nu}$', fontsize=fontsize)
    ax[0,2].set_yscale('log')
    ax[0,2].axhline(y=1.0, **c.AX_LINE_DEFAULTS)


    ax[1,0].set_ylabel(r'$n_e$ / m$^{-3}$', fontsize=fontsize)
    ax[1,0].set_yscale('log')
    ax[1,1].set_ylabel(r'$I_{sat}$ / A', fontsize=fontsize)
    ax[1,2].set_ylabel(r'$a$', fontsize=fontsize)
    ax[1,0].set_xlabel(r'Tilt / $^{\circ}$', fontsize=fontsize)
    ax[1,1].set_xlabel(r'Tilt / $^{\circ}$', fontsize=fontsize)
    ax[1,2].set_xlabel(r'Tilt / $^{\circ}$', fontsize=fontsize)

    ax[1,2].set_yscale('log')
    ax[1,2].set_xlim(-0.5, 10.5)
    # ax[1,2].set_xlim(91.5, 79.5)
    ax[1,2].set_ylim(2e-3, 1e0)

#     fig.set_constrained_layout_pads(hspace=0.0, wspace=3.0)
    fig.legend(handles, labels, loc='center left', bbox_to_anchor=(0.9,0.5))
    fig.suptitle('Anglescan Fit Parameters for {} Plasma at {}'.format(*plasma_settings))
    
    return ax

In [17]:
def plot_paper_tiltscan_fitparams_4(plasma_settings, ax=None, phi_temp_fl=False, probes=('S', 'L'), 
                                    fontsize=18):
    if ax is None:
        fig, ax = plt.subplots(2, 2, sharex=True, 
#                                constrained_layout=True,
#                                figsize=[8,6])
                               figsize=[10,7])
    else:
        fig = ax[0,0].figure
        
    ds = fit_datasets[plasma_settings]
    
    if phi_temp_fl:
        ds['d_temp_phi'] = np.abs(ds['temp_phi'] - ds['temp_fit_phi'])

    
    labels = []
    handles = []
    fp_markers = ['.', 's']

    for i, probe in enumerate(probes):
        probe = probe[0]

        ds_probe = ds.sel(probe=probe)

        probe_dims = magnum_probes[probe_label]
        ave_ts_probe_temps, ave_ts_probe_denss = interpolate_ts_position(ds)
        ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = mgut.interpolate_ts_position(ds, aggregate_dims='probe')
        
        ts_perprobe_data = mgut.interpolate_ts_position(ds_probe, aggregate_dims=None)
        ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = ts_perprobe_data

        dummy_tilt = np.linspace(0.00001, 11.0, 1100)
    #     dummy_tilt = np.linspace(89.99, 78.99, 1101)
        calced_a = lp.calc_sheath_expansion_coeff(
            ave_ts_probe_temps[probe_label][0],
            ave_ts_probe_denss[probe_label][0],
            probe_dims.get_2d_probe_length(),
            probe_dims.g, 
            np.radians(dummy_tilt),
            c_1=0.9,
            c_2=0.6,
        )

        handle = ax[0][0].errorbar('tilt', 'temp', yerr='d_temp', data=ds_probe, label=probe, 
                                   color=probe_colours[probe], marker=probe_markers[probe_label], mfc='none',
                                   linestyle='none')
        labels.append(probe)
        handles.append(handle)
        if phi_temp_fl:
            label = r'$T_e$ from $\phi$ on {}'.format(probe)
            handle = ax[0][0].errorbar('tilt', 'temp_phi', yerr='d_temp_phi', data=ds_probe, mfc='none', 
                                       label=label, linestyle='none', marker='^', color=probe_colours[probe])
            labels.append(label)
            handles.append(handle)
        
#         ds_probe['dens_scaled']  = ds_probe['dens'] / 0.6
        ax[1][0].errorbar('tilt', 'dens', yerr='d_dens', data=ds_probe, 
                          marker=probe_markers[probe_label], mfc='none',
                          linestyle='none', color=probe_colours[probe])
        ax[1][1].errorbar('tilt', 'isat', yerr='d_isat', data=ds_probe, marker=probe_markers[probe_label], 
                          mfc='none', linestyle='none', color=probe_colours[probe])
        ax[0][1].errorbar('tilt', 'a', yerr='d_a', data=ds_probe, marker=probe_markers[probe_label], mfc='none',
                          linestyle='none', color=probe_colours[probe])
        
        calced_a_label = r'$a_{{{probe}, calc}}$'.format(probe=probe)
        handle = ax[0][1].errorbar(dummy_tilt, calced_a, label=calced_a_label, fmt='-', color=probe_colours[probe], 
                                   linewidth=0.8, alpha=0.6)
        labels.append(calced_a_label)
        handles.append(handle)

        # Plot the ts values at the probe positions using interpolaton
        handle = ax[0][0].errorbar(ds_probe['tilt'].values, ts_probe_temps[probe_label][:,0],
                                   yerr=ts_probe_d_temps[probe_label][:,0],
                                   color=probe_colours[probe], linewidth=1, linestyle='dashed', 
                                   label=f'TS at {probe}')
        ax[1][0].errorbar(ds_probe['tilt'].values, ts_probe_denss[probe_label][:,0],
                          yerr=ts_probe_d_denss[probe_label][:,0],
                          color=probe_colours[probe], linewidth=1, linestyle='dashed')

        labels.append(f'TS at {probe}')
        handles.append(handle)    


    ax[0,0].set_ylabel('$T_e$  [eV]', fontsize=fontsize)
    ax[1,0].set_ylabel(r'$n_e$  [m$^{-3}$]', fontsize=fontsize)
    ax[1,0].set_yscale('log')
    ax[0,1].set_ylabel(r'$a$', fontsize=fontsize)
    ax[1,1].set_ylabel(r'$I_{sat}$   [A]', fontsize=fontsize)
    
    ax[1,0].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)
    ax[1,1].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)
    

    ax[0,1].set_yscale('log')
    ax[0,1].set_xlim(-0.5, 10.5)
    # ax[1,2].set_xlim(91.5, 79.5)
    ax[0,1].set_ylim(1e-3, 1e0)

#     fig.set_constrained_layout_pads(hspace=0.1, wspace=0.1)
    fig.tight_layout(pad=0.4, rect=[0, 0, 0.84, 1])
    fig.legend(handles, labels, loc='center left', bbox_to_anchor=(0.85,0.5))
#     fig.suptitle('Anglescan Fit Parameters for {} Plasma at {}'.format(*plasma_settings))
    
    return ax

In [39]:
def plot_paper_tiltscan_fitparams_3(plasma_settings, ax=None, probes=('S', 'L'), fontsize=18, 
                                    orientation='v', a_ylim=[1e-3, 1e0], ne_ylim=None):
    if orientation[0] == 'v':
        layout = [3]
        figsize = [4.5,9.5]
    elif orientation[0] == 'h':
        layout = [1, 3]
        figsize = [12.5,4.75]
    else:
        raise ValueError('Orientation must be one of [h(orizontal), v(ertical)]')
    
    if ax is None:
        fig, ax = plt.subplots(*layout, sharex=True, 
#                                constrained_layout=True,
                               figsize=figsize)
    else:
        fig = ax[0].figure
        
    ds = fit_datasets[plasma_settings]
    ds['ts_d_temperature'] = ds['ts_temperature'] * mean_percentage_temp_ts
    ds['ts_d_density'] = ds['ts_density'] * mean_percentage_dens_ts
    
    labels = []
    handles = []
    fp_markers = ['.', 's']

    for i, probe in enumerate(probes):
        probe_label = probe[0]

        ds_probe = ds.sel(probe=probe)
#         ds['ts_d_temperature'] = ds['ts_temperature'] * mean_percentage_temp_ts

        probe_dims = magnum_probes[probe_label]
        ave_ts_probe_temps, ave_ts_probe_denss = interpolate_ts_position(ds)
        
        ts_perprobe_data = mgut.interpolate_ts_position(ds_probe, aggregate_dims=None)
        ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = ts_perprobe_data

        dummy_tilt = np.linspace(0.00001, 11.0, 1100)
    #     dummy_tilt = np.linspace(89.99, 78.99, 1101)
        calced_a = lp.calc_sheath_expansion_coeff(
            ave_ts_probe_temps[probe_label][0],
            ave_ts_probe_denss[probe_label][0],
            probe_dims.get_2d_probe_length(),
            probe_dims.g, 
            np.radians(dummy_tilt),
#             c_1=0.9,
            c_1=60,
            c_2=0.6,
        )
        probe_a = lp.calc_sheath_expansion_coeff(
            ds_probe['temp'],
            ds_probe['dens'],
            probe_dims.get_2d_probe_length(),
            probe_dims.g, 
            ds_probe['tilt'],
#             c_1=0.9,
            c_1=60,
            c_2=0.6,
        )

        ax[0].errorbar('tilt', 'dens', yerr='d_dens', data=ds_probe, linestyle='none',
                       label=probe, color=probe_colours[probe], marker=probe_markers[probe_label], mfc='none')
#         labels.append(probe)
#         handles.append(handle)
        ax[1].errorbar('tilt', 'temp', yerr='d_temp', data=ds_probe, label=probe, linestyle='none', 
                       color=probe_colours[probe], marker=probe_markers[probe_label], mfc='none')
        
        # Plot the ts values at the probe positions using interpolaton
        ax[0].errorbar(ds_probe['tilt'].values, ts_probe_denss[probe_label][:,0], 
                       label=f'TS at {probe}', yerr=ts_probe_d_denss[probe_label][:,0],
                       color=probe_colours[probe], linewidth=1, linestyle='dashed')
        ax[1].errorbar(ds_probe['tilt'].values, ts_probe_temps[probe_label][:,0], 
                       label=f'TS at {probe}', yerr=ts_probe_d_temps[probe_label][:,0],
                       color=probe_colours[probe], linewidth=1, linestyle='dashed')
#         labels.append(f'TS at {probe}')
#         handles.append(handle)  
        
        ax[2].errorbar('tilt', 'a', yerr='d_a', data=ds_probe, linestyle='none', color=probe_colours[probe],
                       marker=probe_markers[probe_label], mfc='none', label=probe)
#         ax[2].plot(ds_probe['tilt'], probe_a, linestyle='none', color=probe_colours[probe],
#                    marker=probe_markers[probe_label], mfc='none', label=probe)
        
        
        calced_a_label = r'$a_{{{probe}, calc}}$'.format(probe=probe)
        ax[2].errorbar(dummy_tilt, calced_a, label=calced_a_label, fmt='-', 
                       color=probe_colours[probe], linewidth=0.8, alpha=0.6)
#         labels.append(calced_a_label)
#         handles.append(handle)

          


    ax[0].set_ylabel(r'$n_e$  [m$^{-3}$]', fontsize=fontsize)
    ax[0].set_yscale('log')
    ax[1].set_ylabel('$T_e$  [eV]', fontsize=fontsize)
    ax[2].set_ylabel(r'$a$', fontsize=fontsize)
    ax[2].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)
    if orientation[0] == 'h':
        for i in range(2):
            ax[i].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)
        
#     ax[1].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)

    ax[2].set_yscale('log')
    ax[2].set_xlim(-0.5, 10.5)
    ax[2].set_ylim(*a_ylim)
    
    if ne_ylim:
        ax[0].set_ylim(*ne_ylim)

#     fig.set_constrained_layout_pads(hspace=0.1, wspace=0.1)
    fig.tight_layout(pad=0.4) #, rect=[0, 0, 0.8, 1])
#     fig.legend(handles, labels, loc='center left', bbox_to_anchor=(0.75,0.5))
    for axis in ax:
        axis.legend()
#     fig.suptitle('Anglescan Fit Parameters for {} Plasma at {}'.format(*plasma_settings))
    
    return ax

In [19]:
def plot_paper_tiltscan_fitparams_2(plasma_settings, ax=None, probes=('S', 'L'), fontsize=18):
    if ax is None:
        fig, ax = plt.subplots(2, sharex=True, 
#                                constrained_layout=True,
                               figsize=[5,8])
    else:
        fig = ax[0].figure
        
    ds = fit_datasets[plasma_settings]
    
    labels = []
    handles = []

    for i, probe in enumerate(probes):
        probe_label = probe[0]

        ds_probe = ds.sel(probe=probe)

        probe_dims = magnum_probes[probe_label]
        ave_ts_probe_temps, ave_ts_probe_denss = interpolate_ts_position(ds)
        ts_perprobe_data = mgut.interpolate_ts_position(ds, aggregate_dims='probe')
        
        ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = ts_perprobe_data

        dummy_tilt = np.linspace(0.00001, 11.0, 1100)
    #     dummy_tilt = np.linspace(89.99, 78.99, 1101)
        calced_a = lp.calc_sheath_expansion_coeff(
            ave_ts_probe_temps[probe_label][0],
            ave_ts_probe_denss[probe_label][0],
            probe_dims.get_2d_probe_length(),
            probe_dims.g, 
            np.radians(dummy_tilt),
            c_1=0.9,
            c_2=0.6,
        )

        handle = ax[0].errorbar('tilt', 'dens', yerr='d_dens', data=ds_probe, fmt='.', 
                                   color=probe_colours[probe])
        labels.append(probe)
        handles.append(handle)
        
        # Plot the ts values at the probe positions using interpolaton
        handle = ax[0].errorbar(ds_probe['tilt'].values, ts_probe_denss[probe_label][:,0], 
                                   color=probe_colours[probe], linewidth=1, linestyle='dashed')

        labels.append(f'TS at {probe}')
        handles.append(handle)  
        
        ax[1].errorbar('tilt', 'a', yerr='d_a', data=ds_probe, fmt='.', color=probe_colours[probe])
        
        calced_a_label = r'$a_{{{probe}, calc}}$'.format(probe=probe)
        handle = ax[1].errorbar(dummy_tilt, calced_a, label=calced_a_label, fmt='-', 
                                color=probe_colours[probe], linewidth=0.8, alpha=0.6)
        labels.append(calced_a_label)
        handles.append(handle)

          


    ax[0].set_ylabel(r'$n_e$  [m$^{-3}$]', fontsize=fontsize)
    ax[0].set_yscale('log')
    ax[1].set_ylabel(r'$a$', fontsize=fontsize)
    
    ax[1].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)
    ax[1].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)
    

    ax[1].set_yscale('log')
    ax[1].set_xlim(-0.5, 10.5)
    # ax[1,2].set_xlim(91.5, 79.5)
    ax[1].set_ylim(2e-3, 1e0)

#     fig.set_constrained_layout_pads(hspace=0.1, wspace=0.1)
    fig.tight_layout(pad=0.4) #, rect=[0, 0, 0.8, 1])
    fig.legend(handles, labels, loc='center left', bbox_to_anchor=(0.75,0.5))
#     fig.suptitle('Anglescan Fit Parameters for {} Plasma at {}'.format(*plasma_settings))
    
    return ax

In [20]:
def plot_paper_tiltscan_fitparams_extra(plasma_settings, ax=None, phi_temp_fl=False, probes=('S', 'L'), 
                                        fontsize=18, mu=1, Z=1):
    if ax is None:
        fig, ax = plt.subplots(2, 3, sharex=True, 
#                                constrained_layout=True,
                               figsize=[8,6])
    else:
        fig = ax[0,0].figure
        
    ds = fit_datasets[plasma_settings]
    
    if phi_temp_fl:
        ds['d_temp_phi'] = np.abs(ds['temp_phi'] - ds['temp_fit_phi'])

    
    labels = []
    handles = []

    for i, probe in enumerate(probes):
        probe_label = probe[0]

        ds_probe = ds.sel(probe=probe)

        probe_dims = magnum_probes[probe_label]
        ave_ts_probe_temps, ave_ts_probe_denss = interpolate_ts_position(ds)
        ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = mgut.interpolate_ts_position(ds, aggregate_dims='probe')

        dummy_tilt = np.linspace(0.00001, 11.0, 1100)
    #     dummy_tilt = np.linspace(89.99, 78.99, 1101)
        calced_a = lp.calc_sheath_expansion_coeff(
            ave_ts_probe_temps[probe_label][0],
            ave_ts_probe_denss[probe_label][0],
            probe_dims.get_2d_probe_length(),
            probe_dims.g, 
            np.radians(dummy_tilt),
            c_1=0.9,
            c_2=0.6,
        )

        handle = ax[0][0].errorbar('tilt', 'temp', yerr='d_temp', data=ds_probe, label=probe, fmt='.', 
                                   color=probe_colours[probe])
        labels.append(probe)
        handles.append(handle)
        if phi_temp_fl:
            label = r'$T_e$ from $\phi$ on {}'.format(probe)
            handle = ax[0][0].errorbar('tilt', 'temp_phi', yerr='d_temp_phi', data=ds_probe, mfc='none', 
                                       label=label, linestyle='none', marker='^', color=probe_colours[probe])
            labels.append(label)
            handles.append(handle)
        
#         ds_probe['dens_scaled']  = ds_probe['dens'] / 0.6
        ax[1][0].errorbar('tilt', 'dens', yerr='d_dens', data=ds_probe, fmt='.', color=probe_colours[probe])
        ax[1][1].errorbar('tilt', 'isat', yerr='d_isat', data=ds_probe, fmt='.', color=probe_colours[probe])
        ax[0][1].errorbar('tilt', 'a', yerr='d_a', data=ds_probe, fmt='.', color=probe_colours[probe])
        
        calced_a_label = r'$a_{{{probe}, calc}}$'.format(probe=probe)
        handle = ax[0][1].errorbar(dummy_tilt, calced_a, label=calced_a_label, fmt='-', color=probe_colours[probe], 
                                   linewidth=0.8, alpha=0.6)
        labels.append(calced_a_label)
        handles.append(handle)
        
        alt_calced_a = lp.calc_sheath_expansion_coeff(
            ds_probe['temp'],
            ds_probe['dens'],
            probe_dims.get_2d_probe_length(),
            probe_dims.g, 
            np.radians(ds_probe['tilt']),
            c_1=0.9,
            c_2=0.6,
        )
        ax[0][1].errorbar(ds_probe['tilt'], alt_calced_a, label=calced_a_label, marker='s', color=probe_colours[probe])

        # Plot the ts values at the probe positions using interpolaton
        handle = ax[0][0].errorbar(ds_probe['tilt'].values, ts_probe_temps[probe_label][:,0], yerr=0, 
                                   color=probe_colours[probe], linewidth=1, linestyle='dashed', 
                                   label=f'TS at {probe}')
        ax[1][0].errorbar(ds_probe['tilt'].values, ts_probe_denss[probe_label][:,0], 
                          color=probe_colours[probe], linewidth=1, linestyle='dashed')

        labels.append(f'TS at {probe}')
        handles.append(handle)    
        
#         larmor = (102 * np.sqrt(mu * ts_probe_temps[probe_label][:, 0]) / (Z * plasma_settings[0])) 
        larmor = lp.ion_larmor_radius(ts_probe_temps[probe_label][:, 0], plasma_settings[0], mu=mu, Z=Z)
        ax[0,2].plot(ds_probe['tilt'].values, larmor)
        ax[1,2].plot(ds_probe['tilt'].values, probe_dims.get_2d_collection_length(np.radians(ds_probe['tilt'].values)) / larmor)


    ax[0,0].set_ylabel('$T_e$  [eV]', fontsize=fontsize)
    ax[1,0].set_ylabel(r'$n_e$  [m$^{-3}$]', fontsize=fontsize)
    ax[1,0].set_yscale('log')
    ax[0,1].set_ylabel(r'$a$', fontsize=fontsize)
    ax[1,1].set_ylabel(r'$I_{sat}$   [A]', fontsize=fontsize)
    
    ax[1,0].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)
    ax[1,1].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)
    

    ax[0,1].set_yscale('log')
    ax[0,1].set_xlim(-0.5, 10.5)
#     ax[1,2].set_xlim(91.5, 79.5)
    ax[0,1].set_ylim(2e-3, 1e0)

#     fig.set_constrained_layout_pads(hspace=0.1, wspace=0.1)
    fig.tight_layout(pad=0.4, rect=[0, 0, 0.84, 1])
    fig.legend(handles, labels, loc='center left', bbox_to_anchor=(0.85,0.5))
#     fig.suptitle('Anglescan Fit Parameters for {} Plasma at {}'.format(*plasma_settings))
    
    return ax

In [21]:
def plot_coarse_anglescan_ivs(plasma_settings, probe='S', ax=None, ):
    if ax is None:
        fig, ax = plt.subplots(constrained_layout=True)
    else: 
        fig = ax.figure
#     markers = ['.', 'x', '+', '^', '*', 'o']
#     tableau_palette = ['red', 'blue', 'green', 'orange', 'purple', 'black']
    
    fit_ds = fit_datasets[plasma_settings]
    fitter = fts.FullIVFitter()

    coarse_s_ds = fit_ds.sel(probe=probe).isel(tilt=slice(0,11,2))
    for i, tilt in enumerate(coarse_s_ds.tilt.values):
        ds = coarse_s_ds.sel(tilt=tilt)
    #     coarse_s_ds.set_coords('voltage')['current'].plot.line(hue='tilt', x='voltage', ax=ax)
        ax.errorbar(ds['voltage'].values, -ds['current'].values, yerr=ds['d_current'], 
                    marker=markers[i], mfc='none', color=cb_palette[i], linestyle='none', 
                    label=r'${:.2g}^{{\circ}}$'.format(np.abs(tilt)))
#         ax.plot(ds['voltage'].values, fitter.fit_function(ds['voltage'].values, ))
        ax.set_ylabel('Probe Current [A]')
        ax.set_xlabel('Probe Voltage [V]')
        ax.axhline(y=0, **c.AX_LINE_DEFAULTS)
        ax.legend(title=r'$\theta$', loc='lower left')
        
    fig.suptitle('IV Characteristics for {}T {} Plasma'.format(*plasma_settings))
    ax.set_title(probe)
    
    return ax

DEFAULT_LABEL_POS = {
    'x0': 0.48, 
    'x1': 0.95,
    'y0': 0.93,
    'y1': 0.46
}
   
def plot_coarse_anglescan_ivs_multiprobe(plasma_settings, probes=('L', 'S', 'B', 'R'), sharey=False,
                                         label_pos=DEFAULT_LABEL_POS):
    fig, ax = plt.subplots(2, 2, sharex=True, sharey=sharey, figsize=[9,8])
    
    plot_coarse_anglescan_ivs(plasma_settings, probe=probes[0], ax=ax[0,0])
    plot_coarse_anglescan_ivs(plasma_settings, probe=probes[1], ax=ax[0,1])
    plot_coarse_anglescan_ivs(plasma_settings, probe=probes[2], ax=ax[1,0])
    plot_coarse_anglescan_ivs(plasma_settings, probe=probes[3], ax=ax[1,1])
    
    ax[1,1].set_ylabel('')
    ax[0,1].set_ylabel('')
    ax[0,0].set_xlabel('')
    ax[0,1].set_xlabel('')
    ax[0,0].set_title('')
    ax[0,1].set_title('')
    ax[1,0].set_title('')
    ax[1,1].set_title('')
    fig.suptitle('')
    
    fig.tight_layout() #rect=[0, 0, 0.95, 1])
    fig.text(label_pos['x0'], label_pos['y0'], probes[0], ha='center', fontsize=18)
    fig.text(label_pos['x1'], label_pos['y0'], probes[1], ha='center', fontsize=18)
    fig.text(label_pos['x0'], label_pos['y1'], probes[2], ha='center', fontsize=18)
    fig.text(label_pos['x1'], label_pos['y1'], probes[3], ha='center', fontsize=18)
    
def plot_coarse_anglescan_ivs_twoprobe(plasma_settings, probes=['S', 'L'], sharey=False):
    fig, ax = plt.subplots(2, sharex=True, figsize=[5,8], 
                           sharey=sharey)
    
    for i, probe in enumerate(probes):
        plot_coarse_anglescan_ivs(plasma_settings, probe=probe, ax=ax[i])
#     ax[1].set_ylabel('')
    ax[0].set_xlabel('')
    ax[0].set_title('')
    ax[1].set_title('')
    fig.suptitle('')
    
    fig.tight_layout() #rect=[0, 0, 0.95, 1])
    fig.text(0.9, 0.93, probes[0], ha='center', fontsize=18)
    fig.text(0.9, 0.46, probes[1], ha='center', fontsize=18)
    
    
def plot_coarse_anglescan_ivs_twoprobe_horiz(plasma_settings, probes=['S', 'L'], sharey=False):
    fig, ax = plt.subplots(1, 2, sharex=True, figsize=[9, 4.75], 
                           sharey=sharey)
    
    for i, probe in enumerate(probes):
        plot_coarse_anglescan_ivs(plasma_settings, probe=probe, ax=ax[i])
    ax[1].set_ylabel('')
#     ax[0].set_xlabel('')
    ax[0].set_title('')
    ax[1].set_title('')
    fig.suptitle('')
    
    fig.tight_layout() #rect=[0, 0, 0.95, 1])
    fig.text(0.49, 0.9, probes[0], ha='center', fontsize=18)
    fig.text(0.95, 0.9, probes[1], ha='center', fontsize=18)

In [24]:
def plot_ts_profiles(plasma_settings, probes=['S', 'L', 'B', 'R'], shot_index={'tilt':10}):
    fig, ax = plt.subplots(2, sharex=True)
    
    shifted_probes = magnum_probes.probe_position.copy()
    for probe in magnum_probes.probe_position.keys():
        shifted_probes[probe] -= 1.75
    
    fit_ds = fit_datasets[plasma_settings]
    fitter = fts.FullIVFitter()

    ts_ds = fit_ds.sel(probe=probes[0], **shot_index)
    
    ax[0].errorbar('ts_radial_pos', 'ts_temperature', yerr='ts_d_temperature', data=ts_ds)
    ax[1].errorbar('ts_radial_pos', 'ts_density', yerr='ts_d_density', data=ts_ds)

    for probe, pos in shifted_probes.items():
        probe = probe.upper()
    #     pos -= 10
        ax[0].axvline(x=pos, linestyle='dashed', linewidth=0.75, color=probe_colours[probe], label=probe)
        ax[1].axvline(x=pos, linestyle='dashed', linewidth=0.75, color=probe_colours[probe], label=probe)
    ax[0].legend()
    ax[1].legend()
    
plot_ts_profiles(anglescan_plset)

In [40]:
anglescan_plset = (0.8, 'H')
# anglescan_plset = (1.2, 'H')
# anglescan_plset = (1.5, 'H')
# anglescan_plset = (0.8, 'He')
# anglescan_plset = (1.2, 'He')
# plot_paper_tiltscan_fitparams_4(anglescan_plset, probes=('S', 'L'), fontsize=14)
plot_paper_tiltscan_fitparams_3(anglescan_plset, probes=('S', 'L'), fontsize=14, orientation='h')
# plot_coarse_anglescan_ivs_multiprobe(anglescan_plset, label_pos={'x0': 0.46, 'x1': 0.95, 'y0': 0.93, 'y1': 0.46})
# plot_coarse_anglescan_ivs_twoprobe(anglescan_plset, probes=['B', 'R'])
# plot_coarse_anglescan_ivs_twoprobe_horiz(anglescan_plset, sharey=False, probes=['B', 'R'])

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/computation.py:604: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


l 6
s -4
b -14
r -24
l 6
s -4
b -14
r -24


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/computation.py:604: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f1ff4532b00>,
      dtype=object)

In [52]:


fig, ax = plt.subplots(1, 2, figsize=[9, 4.75])

single_anglescan_plset = (1.2, 'H')
plot_coarse_anglescan_ivs(single_anglescan_plset, probe='R', ax=ax[0])

single_anglescan_plset = (1.2, 'He')
plot_coarse_anglescan_ivs(single_anglescan_plset, probe='B', ax=ax[1])

ax[0].set_title('')
ax[1].set_title('')
fig.text(0.46, 0.9, 'R', ha='center', fontsize=18)
fig.text(0.95, 0.9, 'B', ha='center', fontsize=18)

fig.suptitle('')
fig.tight_layout()

In [64]:
fig, ax = plt.subplots(3, 2, sharex=True, figsize=[9,10])

ne_ylim = [1e16, 3e20]
plot_paper_tiltscan_fitparams_3(anglescan_plset, probes=('S', 'L'), fontsize=14, orientation='v', ax=ax[:,0])
plot_paper_tiltscan_fitparams_3(anglescan_plset, probes=('B', 'R'), fontsize=14, orientation='v', ax=ax[:,1], 
                                a_ylim=[1e-3, 5e0], ne_ylim=ne_ylim)


fig.tight_layout()

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/computation.py:604: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


l 6
s -4
b -14
r -24
l 6
s -4
b -14
r -24


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/computation.py:604: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/computation.py:604: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/na

l 6
s -4
b -14
r -24
l 6
s -4
b -14
r -24


In [34]:
def plot_fit_iv(plasma_settings, ax=None, probe='S', iv_indices={'tilt':10}):
    if ax is None:
        fig, ax = plt.subplots(figsize=[5, 4.75])
    else: 
        fig = ax.figure
    
    fit_ds = fit_datasets[plasma_settings].sel(time=slice(None, None, 100))
    fitter = fts.FullIVFitter()
    
    for i, (index, value) in enumerate(iv_indices.items()):
        ds = fit_ds.sel(probe=probe).sel({index: value})
        ds['current'] = -ds['current']
        print(ds)
        
        iv_data = iv.IVData.from_dataset(ds)
        shot_fit = iv_data.multi_fit(**{'mode': 3, 'trimming_vals': (0.16, 0.16, 0.02), 'sat_region': -25})
        fit_dens = magnum_probes.probe_s.get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=np.radians(ds.tilt.values))[0]
        
        ax.errorbar(ds['voltage'].values, ds['current'].values, yerr=ds['d_current'], 
                    marker=markers[i], mfc='none', color=probe_colours[probe], alpha=0.5,
                    linestyle='none', zorder=-3)
        ax.plot(*shot_fit.get_fit_plottables(), '-', color=probe_colours[probe],
                label=r'$T_e$={:.3g}, $n_e$={:.3g}, $\chi^2_{{\nu}}$ = {:.3g}'
                .format(shot_fit.get_temp(), fit_dens, shot_fit.reduced_chi2))
        
        ax.set_ylabel('Probe Current [A]')
        ax.set_xlabel('Probe Voltage [V]')
        ax.axhline(y=0, **c.AX_LINE_DEFAULTS)
        ax.legend(loc='lower left')
#         ax.legend(title=r'$\theta$', loc='lower left')

#         fig.suptitle('IV Characteristics for {}T {} Plasma'.format(*plasma_settings))
    
    fig.tight_layout() #rect=[0, 0, 0.95, 1])
    
    

plot_fit_iv(anglescan_plset, probe='L', iv_indices={'tilt':10})

<xarray.Dataset>
Dimensions:                 (time: 50, ts_radial_pos: 46)
Coordinates:
    probe                   <U1 'L'
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    float64 10.0
Dimensions without coordinates: time
Data variables:
    voltage                 (time) float64 -89.75 -88.44 ... -14.59 -14.35
    current                 (time) float64 0.1655 0.1676 ... -0.5599 -0.6824
    shot_time               (time) float64 4.333 4.333 4.333 ... 4.333 4.333
    start_time              float64 4.33
    d_current               (time) float64 0.002088 0.002042 ... 0.004808
    std_current             (time) float64 0.07622 0.07453 ... 0.1632 0.1755
    shot_number             float64 498.0
    ts_density              (ts_radial_pos) float64 nan nan nan ... nan nan
    ts_temperature          (ts_radial_pos) float64 nan nan nan ... nan nan
    ts_d_density            (ts_radial_pos) float64 nan nan nan ... nan nan
    ts_d_t

In [22]:
import importlib
importlib.reload(mgut)
importlib.reload(lp)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

In [23]:
# for probe in probes:
#     plot_probe_anglescans(probe, fit_datasets)

In [24]:
# plot_probe_anglescans('S', fit_datasets)

## Getting the TS Dataset to Plot Errorbars Properly

In [27]:
full_ts_ds = xr.open_dataset('full_ts_dataset.nc')

In [28]:
shot_number_ds = meta_data_ds.dropna('shot_number', subset=['adc_index']).swap_dims({'shot_number':'adc_index'}).sel(adc_index=indices_08T_H)
shot_numbers = shot_number_ds.shot_number
shot_number_ds

<xarray.Dataset>
Dimensions:                 (adc_index: 22, ts_radial_pos: 46)
Coordinates:
    shot_number             (adc_index) int32 36 37 38 39 40 ... 54 55 56 57 58
    ts_number               (adc_index) float64 ...
    ts_timestamp            (adc_index) float64 ...
    ts_time                 (adc_index) datetime64[ns] ...
  * adc_index               (adc_index) float64 41.0 42.0 43.0 ... 62.0 63.0
    adc_time                (adc_index) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (adc_index) object ...
    ts_density              (adc_index, ts_radial_pos) float64 ...
    ts_temperature          (adc_index, ts_radial_pos) float64 ...
    ts_d_density            (adc_index, ts_radial_pos) float64 ...
    ts_d_temperature        (adc_index, ts_radial_pos) float64 ...
    adc_folder              (adc_index) object ...
    adc_calibration_index   (adc_index) object ...
    adc_4_probe             (adc_index) object ...
    adc_5_probe             (adc_index) object ...
    ts_temp_max             (adc_index) float64 ...
    ts_dens_max             (adc_index) float64 ...
    shot_end_time           (adc_index) datetime64[ns] ...
    ts_fl                   (adc_index) bool ...
    shot_time               (adc_index) datetime64[ns] ...
    shot_source_current     (adc_index) float64 ...
    shot_b_field            (adc_index) float64 ...
    shot_tilt               (adc_index) float64 ...
    shot_target_pos         (adc_index) float64 ...
    shot_target_rot         (adc_index) float64 ...
    shot_target_pumpspeed   (adc_index) int32 ...
    shot_source_pumpspeed   (adc_index) int32 ...
    shot_heating_pumpspeed  (adc_index) int32 ...
    shot_source_pos         (adc_index) float64 ...
    shot_deuterium_gf       (adc_index) float64 ...
    shot_hydrogen_gf        (adc_index) float64 ...
    shot_helium_gf          (adc_index) float64 ...
    shot_hydrogen_puffed    (adc_index) float64 ...
    shot_helium_puffed      (adc_index) float64 ...
    adc_timestamp           (adc_index) float64 ...
    adc_4_coax              (adc_index) float64 ...
    adc_4_shunt_resistance  (adc_index) float64 ...
    adc_5_coax              (adc_index) float64 ...
    adc_5_shunt_resistance  (adc_index) float64 ...
    adc_calibration_fl      (adc_index) float64 ...
    shot_fl                 (adc_index) float64 ...
    acquisition_length      (adc_index) float64 ...
    adc_end_time            (adc_index) datetime64[ns] ...
    adc_end_timestamp       (adc_index) float64 ...
    adc_freqs               (adc_index) float64 ...

In [29]:
meta_data_ds.sel(shot_number=36)

<xarray.Dataset>
Dimensions:                 (ts_radial_pos: 46)
Coordinates:
    shot_number             int32 36
    ts_number               float64 ...
    ts_timestamp            float64 ...
    ts_time                 datetime64[ns] ...
    adc_index               float64 41.0
    adc_time                datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            object ...
    ts_density              (ts_radial_pos) float64 ...
    ts_temperature          (ts_radial_pos) float64 ...
    ts_d_density            (ts_radial_pos) float64 ...
    ts_d_temperature        (ts_radial_pos) float64 ...
    adc_folder              object ...
    adc_calibration_index   object ...
    adc_4_probe             object ...
    adc_5_probe             object ...
    ts_temp_max             float64 ...
    ts_dens_max             float64 ...
    shot_end_time           datetime64[ns] ...
    ts_fl                   bool ...
    shot_time               datetime64[ns] ...
    shot_source_current     float64 ...
    shot_b_field            float64 ...
    shot_tilt               float64 ...
    shot_target_pos         float64 ...
    shot_target_rot         float64 ...
    shot_target_pumpspeed   int32 ...
    shot_source_pumpspeed   int32 ...
    shot_heating_pumpspeed  int32 ...
    shot_source_pos         float64 ...
    shot_deuterium_gf       float64 ...
    shot_hydrogen_gf        float64 ...
    shot_helium_gf          float64 ...
    shot_hydrogen_puffed    float64 ...
    shot_helium_puffed      float64 ...
    adc_timestamp           float64 ...
    adc_4_coax              float64 ...
    adc_4_shunt_resistance  float64 ...
    adc_5_coax              float64 ...
    adc_5_shunt_resistance  float64 ...
    adc_calibration_fl      float64 ...
    shot_fl                 float64 ...
    acquisition_length      float64 ...
    adc_end_time            datetime64[ns] ...
    adc_end_timestamp       float64 ...
    adc_freqs               float64 ...

In [30]:
H_08T_ts = full_ts_ds.sel(ts_time=np.isin(full_ts_ds.shot_number, shot_numbers))
H_08T_ts

<xarray.Dataset>
Dimensions:           (ts_radial_pos: 46, ts_time: 22)
Coordinates:
  * ts_radial_pos     (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    ts_number         (ts_time) int64 ...
    ts_timestamp      (ts_time) int64 ...
  * ts_time           (ts_time) datetime64[ns] 2019-05-29T11:14:59.804525 ... 2019-05-29T12:43:47.398283
    shot_number       (ts_time) int64 36 37 38 39 40 41 43 ... 53 54 55 56 57 58
Data variables:
    ts_density        (ts_time, ts_radial_pos) float64 ...
    ts_d_density      (ts_time, ts_radial_pos) float64 ...
    ts_temperature    (ts_time, ts_radial_pos) float64 ...
    ts_d_temperature  (ts_time, ts_radial_pos) float64 ...

In [31]:
mean_percentage_dens_ts = (H_08T_ts['ts_d_density'] / H_08T_ts['ts_density']).mean('ts_time').values
mean_percentage_temp_ts = (H_08T_ts['ts_d_temperature'] / H_08T_ts['ts_temperature']).mean('ts_time').values

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [33]:

for shot_number in H_08T_ts.shot_number.values[0:1]:
    shot_ds = H_08T_ts.where(H_08T_ts.shot_number == shot_number, drop=True)
    md_shot_ds = meta_data_ds.sel(shot_number=shot_number)
    
    fig, ax = plt.subplots(2, sharex=True)
    fig.suptitle(f'{shot_number} - {md_shot_ds.shot_tilt.values}')
    
    ax[0].errorbar('ts_radial_pos', 'ts_temperature', yerr='ts_d_temperature', data=md_shot_ds, label='averaged')
    ax[1].errorbar('ts_radial_pos', 'ts_density', yerr='ts_d_density', data=md_shot_ds, label='averaged')
    
    for ts_time in shot_ds.ts_time.values:
        ds = shot_ds.sel(ts_time=ts_time)
        
        ax[0].errorbar('ts_radial_pos', 'ts_temperature', yerr='ts_d_temperature', data=ds, label='raw')
        ax[1].errorbar('ts_radial_pos', 'ts_density', yerr='ts_d_density', data=ds, label='raw')

In [29]:
for shot_number in H_08T_ts.shot_number.values[0:5]:
    shot_ds = H_08T_ts.where(H_08T_ts.shot_number == shot_number, drop=True)
    md_shot_ds = meta_data_ds.sel(shot_number=shot_number)
    
    fig, ax = plt.subplots(2, sharex=True)
    fig.suptitle(f'{shot_number} - {md_shot_ds.shot_tilt.values}')
    
    ax[0].plot('ts_radial_pos', 'ts_d_temperature', data=md_shot_ds, label='averaged')
    ax[1].plot('ts_radial_pos', 'ts_d_density', data=md_shot_ds, label='averaged')
    
    ax[0].plot('ts_radial_pos', 'ts_d_temperature', data=shot_ds.std('ts_time'), label='averaged_alt')
    ax[1].plot('ts_radial_pos', 'ts_d_density', data=shot_ds.std('ts_time'), label='averaged_alt')
    
    for ts_time in shot_ds.ts_time.values:
        ds = shot_ds.sel(ts_time=ts_time)
        
        ax[0].plot('ts_radial_pos', 'ts_d_temperature', data=ds, label='raw')
        ax[1].plot('ts_radial_pos', 'ts_d_density', data=ds, label='raw')
        
        ax[0].plot(ds['ts_radial_pos'].values, ds['ts_temperature'].values * mean_percentage_temp_ts, label='estimated')
        ax[1].plot(ds['ts_radial_pos'].values, ds['ts_density'].values * mean_percentage_dens_ts, label='estimated')
        
    for axis in ax:
        axis.legend()
#         axis.set_yscale('log')

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


## Answering Pete's Questions

Pete asked me a few questions following the presentation in an email. Namely the questions were: 
 1. If I use the $T_e$ from the TS in the density calculation, does the larger probe underestimate (compared to TS) more than small probe?
 1. Look at TS profiles with different distances from probes. 
 1. Compare the $\lambda_D$ / $T_e$ / $n_i$ produced from the $I_{sat}$ and from $a$

In [137]:
def plot_temperatures_pete(plasma_settings, ax=None, probes=('S', 'L'), fontsize=18):
    layout = [3]
    figsize = [4.5,9.5]
    
    if ax is None:
        fig, ax = plt.subplots(*layout, sharex=True, 
                               figsize=figsize)
    else:
        fig = ax[0,0].figure
        
    ds = fit_datasets[plasma_settings]
    ds['ts_d_temperature'] = ds['ts_temperature'] * mean_percentage_temp_ts
    ds['ts_d_density'] = ds['ts_density'] * mean_percentage_dens_ts
    
    labels = []
    handles = []
    fp_markers = ['.', 's']

    for i, probe in enumerate(probes):
        probe_label = probe[0]

        ds_probe = ds.sel(probe=probe)
        ds['ts_d_temperature'] = ds['ts_temperature'] * mean_percentage_temp_ts

        probe_dims = magnum_probes[probe_label]
        ave_ts_probe_temps, ave_ts_probe_denss = interpolate_ts_position(ds)
        
        ts_perprobe_data = mgut.interpolate_ts_position(ds_probe, aggregate_dims=None)
        ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = ts_perprobe_data

        dummy_tilt = np.linspace(0.00001, 11.0, 1100)
        calced_a = lp.calc_sheath_expansion_coeff(
            ave_ts_probe_temps[probe_label][0],
            ave_ts_probe_denss[probe_label][0],
            probe_dims.get_2d_probe_length(),
            probe_dims.g, 
            np.radians(dummy_tilt),
            c_1=0.9,
            c_2=0.6,
        )
        probe_a = lp.calc_sheath_expansion_coeff(
            ds_probe['temp'],
            ds_probe['dens'],
            probe_dims.get_2d_probe_length(),
            probe_dims.g, 
            np.radians(ds_probe['tilt']),
            c_1=0.9,
            c_2=0.6,
        ) - ((np.radians(ds_probe['tilt'].values) * -5.168444408771983e-05) + 1.8689974068124587e-06)        
        ts_calced_dens = probe_dims.get_density(ds_probe['isat'], 
                                                ts_probe_temps[probe_label][:,0], 
                                                np.radians(ds_probe['tilt']))

        ax[0].errorbar('tilt', 'dens', yerr='d_dens', data=ds_probe, linestyle='none',
                       label=probe, color=probe_colours[probe], marker=probe_markers[probe_label], mfc='none')

        ax[1].errorbar('tilt', 'temp', yerr='d_temp', data=ds_probe, label=probe, linestyle='none', 
                       color=probe_colours[probe], marker=probe_markers[probe_label], mfc='none')
        
        # Plot the ts values at the probe positions using interpolaton
        ax[0].errorbar(ds_probe['tilt'].values, ts_probe_denss[probe_label][:,0], 
                       label=f'TS at {probe}', yerr=ts_probe_d_denss[probe_label][:,0],
                       color=probe_colours[probe], linewidth=1, linestyle='dashed')
        ax[1].errorbar(ds_probe['tilt'].values, ts_probe_temps[probe_label][:,0], 
                       label=f'TS at {probe}', yerr=ts_probe_d_temps[probe_label][:,0],
                       color=probe_colours[probe], linewidth=1, linestyle='dashed')
        
        ax[0].plot(ds_probe['tilt'].values, ts_calced_dens, label=f'{probe} with TS Temp', 
                   color=probe_colours[probe], linestyle='none', marker='o')

        
        ax[2].errorbar('tilt', 'a', yerr='d_a', data=ds_probe, linestyle='none', color=probe_colours[probe],
                       marker=probe_markers[probe_label], mfc='none', label=probe)        
        
        calced_a_label = r'$a_{{{probe}, calc}}$'.format(probe=probe)
        ax[2].errorbar(dummy_tilt, calced_a, label=calced_a_label, fmt='-', 
                       color=probe_colours[probe], linewidth=0.8, alpha=0.6)
        ax[2].errorbar(ds_probe['tilt'].values, probe_a, label=calced_a_label, fmt='o', 
                       color=probe_colours[probe])
         


    ax[0].set_ylabel(r'$n_e$  [m$^{-3}$]', fontsize=fontsize)
    ax[0].set_yscale('log')
    ax[1].set_ylabel('$T_e$  [eV]', fontsize=fontsize)
    ax[2].set_ylabel(r'$a$', fontsize=fontsize)
    ax[2].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)
        
#     ax[1].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)

    ax[2].set_yscale('log')
    ax[2].set_xlim(-0.5, 10.5)
    ax[2].set_ylim(1e-3, 1e0)


    fig.tight_layout(pad=0.4) #, rect=[0, 0, 0.8, 1])
#     fig.legend(handles, labels, loc='center left', bbox_to_anchor=(0.75,0.5))
    for axis in ax:
        axis.legend()

    
    return ax

In [138]:
def calc_debye_from_a(a, alpha, probe, c_1=0.4, c_2=0.5):
    return (a * (probe.L + probe.g) * np.sqrt(np.sin(alpha))) / (c_1 + (c_2 / np.tan(alpha)))

def calc_debye_from_alt_a(a, alpha, probe, c_1=0.4, c_2=0.5):
    return a * (((((probe.L + probe.g) * np.tan(alpha)) + (probe.L * np.tan(probe.theta_p)) - probe.d_perp) 
                 * np.sqrt(np.sin(alpha))) 
                / (((c_1 * (np.tan(alpha) + np.tan(probe.theta_p))) + c_2)))

def plot_debye_pete(plasma_settings, probes=('S', 'L'), fontsize=18):
    figsize = [4.5, 7]
    fig, ax = plt.subplots(2, sharex=True, figsize=figsize)
    fig2, ax2 = plt.subplots(2, sharex=True, figsize=figsize)
    
    ds = fit_datasets[plasma_settings]
    ds['ts_d_temperature'] = ds['ts_temperature'] * mean_percentage_temp_ts
    ds['ts_d_density'] = ds['ts_density'] * mean_percentage_dens_ts
    
    labels = []
    handles = []
    fp_markers = ['.', 's']

    for i, probe in enumerate(probes):
        probe_label = probe[0]

        ds_probe = ds.sel(probe=probe)
        ds['ts_d_temperature'] = ds['ts_temperature'] * mean_percentage_temp_ts

        probe_dims = magnum_probes[probe_label]
        
        ave_ts_probe_temps, ave_ts_probe_denss = interpolate_ts_position(ds)
        
        ts_perprobe_data = mgut.interpolate_ts_position(ds_probe, aggregate_dims=None)
        ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = ts_perprobe_data
        
        debye_from_a = calc_debye_from_a(ds_probe['a'], np.radians(ds_probe['tilt']), probe_dims)
#         debye_from_a_corr = debye_from_a / ((0.5 * np.radians(ds_probe['tilt'].values)) + 0.16)
        debye_from_a_corr = debye_from_a + ((np.radians(ds_probe['tilt'].values) * -5.168444408771983e-05) + 1.8689974068124587e-06)
#         debye_from_alt_a = calc_debye_from_alt_a(ds_probe['a'], np.radians(ds_probe['tilt']), probe_dims)
        debye_from_tn = lp.debye_length(ds_probe['temp'], ds_probe['dens'])

        ax[i].plot(ds_probe['tilt'].values, debye_from_a, label=r'{} - a'.format(probe), 
                   color=probe_colours[probe], linestyle='none', marker='s', mfc='none')
        ax[i].plot(ds_probe['tilt'].values, debye_from_a_corr, label=r'{} - a'.format(probe), 
                   color=probe_colours[probe], linestyle='none', marker='^', mfc='none')
        ax[i].plot(ds_probe['tilt'].values, debye_from_tn, label=r'{} - $T_e$ and $n_e$'.format(probe), 
                   color=probe_colours[probe], linestyle='none', marker='.', mfc='none')
            
        
        ax2[0].plot(np.radians(ds_probe['tilt'].values), debye_from_tn - debye_from_a, label=r'{}'.format(probe), 
                    color=probe_colours[probe], linestyle='none', marker='s', mfc='none')
        ax2[1].plot(np.radians(ds_probe['tilt'].values), debye_from_a - debye_from_tn, label=r'{}'.format(probe), 
                    color=probe_colours[probe], linestyle='none', marker='s', mfc='none')
        
        sl_fitter = fts.StraightLineFitter()
        fit_data = sl_fitter.fit(np.radians(ds_probe['tilt'].values),  debye_from_tn - debye_from_a)
        fit_data.plot(ax=ax2[0])
        fit_data.print_fit_params()
        
#         ax2[1].axhline(y=1, color=probe_colours[probe], linestyle='--', linewidth=0.75, alpha=0.8)
#         dummy_tilt = np.linspace(0.00001, 11.0, 1100)
#         ax2[1].plot(dummy_tilt, 1/((0.5 * dummy_tilt) + 0.16), label=r'{}'.format(probe), 
#                     color=probe_colours[probe], linestyle='--', marker='', alpha=0.8)
#         ax2[1].plot(dummy_tilt, 1/fit_data.fit_function(dummy_tilt), label=r'{}'.format(probe), 
#                     color=probe_colours[probe], linestyle='-', marker='', alpha=0.8)


    ax[0].set_ylabel(r'$\lambda_D$  [m]', fontsize=fontsize)
    ax[1].set_ylabel(r'$\lambda_D$  [m]', fontsize=fontsize)
    ax[1].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)
        
#     ax[1].set_xlabel(r'$\theta$  [$^{\circ}$]', fontsize=fontsize)

#     ax[2].set_yscale('log')
    ax[0].set_xlim(-0.5, 10.5)
#     ax2[0].set_xlim(-0.5, 10.5)
#     ax[2].set_ylim(1e-3, 1e0)


    fig.tight_layout(pad=0.4) #, rect=[0, 0, 0.8, 1])
    fig2.tight_layout(pad=0.4) #, rect=[0, 0, 0.8, 1])
    for axis in ax:
        axis.legend()
        
    

    
    return ax

In [139]:
plot_temperatures_pete(anglescan_plset, probes=('S', 'L'), fontsize=14)
# plot_debye_pete(anglescan_plset, probes=('S', 'L'), fontsize=14)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


l 6
s -4
b -14
r -24
l 6
s

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


 -4
b -14
r -24


array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f15a8afe2e8>,
      dtype=object)